In [84]:
# importo le librerie
import pandas as pd
import numpy as np
import pulp

# importo le funzioni
from os.path import join
from itertools import product

In [ ]:
# passaggio da eseguire una volta sola

df_source_polimi = pd.read_excel('source_from_polimi.xlsx')

# così è un multiindex con il nome come indice
df_source = pd.DataFrame(df_source_polimi.groupby(['Denominazione','Codice','Sem','CFU','Rating'])['Gruppo'].apply(list))

# ora ritornano dei numeri come indice
df_source.reset_index(inplace=True)

# lo scrivo
df_source.to_excel('source.xlsx')

In [112]:
# PARAMETRI DA SCEGLIERE

# scelta del track
choices = ['MCS','MMF','MST']
track = choices[2]
print(f'You chose {track}. Congratulations!')

CFU_max_sem = 35
CFU_max_tot = 121 # il max di quelli a scelta (obbligatori sono 68)

You chose MST. Congratulations!


In [113]:
PATH_CFU_COSTRAINTS = join('assets','min_CFUs.xlsx')
df_cfu = pd.read_excel(PATH_CFU_COSTRAINTS,index_col=0).dropna()
df_cfu[track] = 0 # aggiungo colonna il gruppo del track con minimo 0 CFU

GROUPS_names = df_cfu.columns.tolist()
min_cfus = df_cfu.loc[track].tolist()

In [114]:
# DATAFRAME CORSI OBBLIGATORI
PATH1 = join('assets','source_'+track+'_1.xlsx')
PATH2 = join('assets','source_'+track+'_2.xlsx')
# comp = compulsory
df_comp1 = pd.read_excel(PATH1).dropna()
df_comp2 = pd.read_excel(PATH2).dropna()
comp1 = len(df_comp1)
comp2 = len(df_comp2)
df_comp = pd.concat([df_comp1, df_comp2]) # unisco i compulsory dei due anni
df_comp.reset_index(inplace=True, drop=True) # reset dell'index 0-4,0-1

df_comp

,Denominazione,Codice,Sem,CFU,Gruppo,Rating
0,REAL AND FUNCTIONAL ANALYSIS,95958,1,8,MST,0
1,STOCHASTIC DYNAMICAL MODELS,54074,1,8,MST,0
2,ALGORITHMS AND PARALLEL COMPUTING,52496,1,10,MST,0
3,MODEL IDENTIFICATION AND DATA ANALYSIS,96297,2,10,MST,0
4,APPLIED STATISTICS,52498,2,10,MST,0
5,BAYESIAN STATISTICS,52499,1,10,MST,0
6,FINAL WORK,97690,2,12,MST,0


In [117]:
df_user = pd.read_excel('source.xlsx',index_col=0).dropna()
df_user['Gruppo'] = df_user['Gruppo'].apply(eval) # serve perché nel file i gruppi sono ['FREE'], mentre vogliamo [FREE]
df_user

,Denominazione,Codice,Sem,CFU,Rating,Gruppo
2,ADVANCED PARTIAL DIFFERENTIAL EQUATIONS,95963,2,8,1.0,"[MTM, FREE]"
3,ADVANCED PROGRAMMING FOR SCIENTIFIC COMPUTING,54073,2,10,3.0,"[CSCL, MTM]"
10,ARTIFICIAL NEURAL NETWORKS AND DEEP LEARNING,54307,1,5,5.0,"[STAT, ING, FREE]"
20,CALCULUS OF VARIATIONS,97673,2,8,1.0,"[MTM, FREE]"
21,COMMUNICATION AND ARGUMENTATION,52582,2,5,1.0,[FREE]
28,COMPUTATIONAL STATISTICS,55700,1,8,4.0,"[MTM, FREE]"
32,CONTINUOUS DYNAMICAL SYSTEMS,55728,2,5,4.0,[MTM]
34,CRITICAL THINKING,55806,2,5,1.0,[FREE]
35,CRITICAL THINKING,55809,2,3,1.0,[FREE]
37,DATA MINING,56892,1,5,4.0,"[STAT, ING, FREE]"


In [120]:
# creo un dataframe per dare corrispondenza codice-denominazione
code_name = pd.concat([df_comp, df_user]).set_index('Codice')

# tolgo le colonne non necessarie
code_name.drop(columns=['Sem','CFU','Gruppo','Rating'],inplace=True)

# NB ci si accede in questo modo
# code_name.loc[56955]['Denominazione']

# rimuovo i doppi indici
code_name = code_name[~code_name.index.duplicated(keep='last')] # specificare quale tenere perché potrebbero a priori avere "Denominazione" diversa, ma non in questo caso perché uguale codice => uguale nome

# converto in un dizionario
code_name = code_name.to_dict()['Denominazione']

#code_name

In [121]:
df_user = df_user.drop(columns=['Denominazione'])
df_comp = df_comp.drop(columns=['Denominazione'])

In [122]:
df = pd.concat([df_comp, df_user]).reset_index(drop=True) # drop serve per far si che il vecchio brutto indice non diventi una colonna

df.head(10)

,Codice,Sem,CFU,Gruppo,Rating
0,95958,1,8,MST,0.0
1,54074,1,8,MST,0.0
2,52496,1,10,MST,0.0
3,96297,2,10,MST,0.0
4,52498,2,10,MST,0.0
5,52499,1,10,MST,0.0
6,97690,2,12,MST,0.0
7,95963,2,8,"[MTM, FREE]",1.0
8,54073,2,10,"[CSCL, MTM]",3.0
9,54307,1,5,"[STAT, ING, FREE]",5.0


In [123]:
dfexplode = df.explode('Gruppo')
dfexplode.head()

,Codice,Sem,CFU,Gruppo,Rating
0,95958,1,8,MST,0.0
1,54074,1,8,MST,0.0
2,52496,1,10,MST,0.0
3,96297,2,10,MST,0.0
4,52498,2,10,MST,0.0


In [124]:
#dfcrosstab = pd.crosstab(dfexplode['Codice'], dfexplode['Gruppo']).ne(0).reset_index()
dfcrosstab = pd.crosstab(dfexplode['Codice'], dfexplode['Gruppo'])
#dfcrosstab.loc[52503]['FREE']
dfcrosstab.head()

Gruppo,CSCL,FREE,ING,MST,MTM,STAT
Codice,,,,,,
51132,0,1,0,0,0,1
52496,0,0,0,1,0,0
52498,0,0,0,1,0,0
52499,0,0,0,1,0,0
52503,0,1,0,0,1,0


In [125]:
dict_base = df.to_dict()

In [126]:
# PARAMETRI
codes    = dict_base['Codice']
semester = dict_base['Sem']
cfus     = dict_base['CFU']
rating   = dict_base['Rating']

In [127]:
# INSIEMI
COURSES = list(codes.keys())
YEARS   = [1, 2]
GROUPS  = list(range(len(GROUPS_names)))

In [128]:
# PROBLEMA
prob = pulp.LpProblem('Piano_di_studi', pulp.const.LpMaximize)

In [129]:
# VARIABILI
x = pulp.LpVariable.dicts('corso-anno-gruppo', (COURSES, YEARS, GROUPS), lowBound=0, upBound=1, cat='Continuous')  # quanto corso i anno j gruppo k
y = pulp.LpVariable.dicts('corso',             (COURSES),                lowBound=0, upBound=1, cat='Binary')      # 1 se corso i è scelto
z = pulp.LpVariable.dicts('corso-anno',        (COURSES, YEARS),         lowBound=0, upBound=1, cat='Binary')      # 1 se corso i all'anno j

In [130]:
# FUNZIONE OBIETTIVO
prob += pulp.lpSum([rating[i]*cfus[i]*x[i][j][k] for (i, j, k) in product(COURSES, YEARS, GROUPS)])

In [131]:
# VINCOLI

# (richiesta) obbligatorietà dei corsi
for i in range(comp1):
    # 5 dovrebbe essere l'indice dell'ultimo gruppo, quello "finto" della track
    prob += x[i][1][GROUPS[-1]] == 1, f"Compulsory course {i} in year 1"

for i in range(comp1, comp1+comp2):
    prob += x[i][2][GROUPS[-1]] == 1, f"Compulsory course {i} in year 2"

# (capacità) max cfu totali
prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i, j, k) in product(COURSES, YEARS, GROUPS)]) <= CFU_max_tot, f"Maximum total CFUs"
#prob += pulp.lpSum([cfus[i]*y[i] for i in COURSES]) <= CFU_max_tot, f"Maximum total CFUs"

# (capacità) max cfu per semestre per anno
for j in YEARS:
    target_groups = [i for i in COURSES if semester[i] == 1]
    prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i,k) in product(target_groups,GROUPS)]) <= CFU_max_sem, f"Maximum CFUs for year {j} semester 1"

    target_groups = [i for i in COURSES if semester[i] == 2]
    prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i,k) in product(target_groups,GROUPS)]) <= CFU_max_sem, f"Maximum CFUs for year {j} semester 2"

# (logico) se un corso è scelto allora z_ij = 1
for (i,j) in product(COURSES,YEARS):
    prob += pulp.lpSum([x[i][j][k] for k in GROUPS]) <= 1000 * z[i][j], f"If course {i} chosen, then z[{i}][{j}]=1"

# (capacità) non posso scegliere un corso in più anni (cioè mettere GAT metà al primo e metà al secondo annno per es.)
for i in COURSES:
    prob += pulp.lpSum([z[i][j] for j in YEARS]) <= 1, f"Course {i} in no more than one year"

# i prossimi due potrebbero essere riassunti in sum... = y[i] ma per discorsi di approssimazione forse meglio così
    
# (logico) se un corso è scelto allora y[i]=1
# (capacità) corso scelto al massimo una sola volta (<=1)
for i in COURSES:
    prob += pulp.lpSum([x[i][j][k] for (j, k) in product(YEARS, GROUPS)]) <= y[i], f"If course {i} chosen, then y[{i}]=1"

# (assegnamento) se y_i=1 allora tutti quanti sopra tutti i gruppi devono sommare a 1 (circa)
for i in COURSES:
    prob += y[i] <= pulp.lpSum([x[i][j][k] for (j, k) in product(YEARS, GROUPS)]), f"If course {i} chosen, then sum of x[{i}][j][k]=1"

# (richiesta) min cfu gruppo (nessun vincolo per l'ultimo che è quello della track)
for k in GROUPS[:-1]:
    prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i,j) in product(COURSES,YEARS)]) >= min_cfus[k], f"Minimum CFUs for group {k}"  # >=/==

# (capacità) posso dare peso a un corso in un gruppo solo è consentito
for (i,k) in product(COURSES,GROUPS):
    prob += pulp.lpSum([x[i][j][k] for j in YEARS]) <= dfcrosstab.loc[codes[i]][GROUPS_names[k]], f"Possibility to choose course {i} in group {k} only if allowed"
    
# dfcrosstab.loc[52503]['FREE']
# dfcrosstab.loc[codes[...]][GROUPS_names[...]]

In [132]:
# RISOLUZIONE
status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/teo/Documents/GitHub/compilatore-piano-di-studi/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/1n/5l6269bd647d957r5df7_hj80000gn/T/d1bdabd293524002a5f8ad05266dd3a3-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/1n/5l6269bd647d957r5df7_hj80000gn/T/d1bdabd293524002a5f8ad05266dd3a3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 330 COLUMNS
At line 3222 RHS
At line 3548 BOUNDS
At line 3969 ENDATA
Problem MODEL has 325 rows, 420 columns and 2471 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 253 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 33 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 147 rows, 141 columns (63 intege

In [133]:
print(f"Status:    {pulp.LpStatus[status]}")
print(f"Objective: {prob.objective.value()}")

Status:    Optimal
Objective: 244.0


In [134]:
#prob.constraints

In [143]:
#prob.writeLP('pulp_problem_description.txt')

In [142]:
# output
# ne servono 2
# - SOMMARIO, quello che uno ha bisogno di sapere, con le variabili z, quale corso in quale anno
# - DETTAGLI, quello da dare a Gregoratti, quale percentuale di quale corso in quale gruppo, con le x

corsi=[]
for (i,j,k) in product(COURSES, YEARS, GROUPS):
    val = x[i][j][k].varValue
    if val and val > 0:
        #codice = codes[i]
        #gruppo = GROUPS_names[k]
        #anno = YEARS[j-1]
        #string = f'{val*100:3.0f}% del corso: {codice}\t\tnel gruppo {gruppo}\t\tnell\'anno {anno}'
        #print(string)
                           
        course_name=code_name[codes[i]]
        semester_name=semester[i]
        cfus_name=cfus[i]
        codes_name=codes[i]
        group_name=GROUPS_names[k]
        year_name = YEARS[j-1]
        percentage = val
        el=[codes_name, course_name, cfus_name, group_name, year_name, semester_name, val]
        corsi.append(el)
        print(f"{val*100:3.0f}% di {course_name} [{group_name}, {cfus_name} CFU] all'anno {year_name} al semestre {semester_name}")
        # :3.0f

piano=pd.DataFrame(corsi, columns=['Codice', 'Denominazione', 'CFU', 'Gruppo', 'Anno', 'Sem','%'])
piano=piano.set_index('Codice')
piano=piano.sort_values(['Anno', 'Sem'], ascending=[True, True])

print(piano)

total_cfus = pulp.lpSum([cfus[i]*x[i][j][k].varValue for (i, j, k) in product(COURSES, YEARS, GROUPS)])
print(f'Total number of CFUs:      {total_cfus}')

for j in YEARS:
    target_groups = [i for i in COURSES if semester[i] == 1]
    cfus_sem_1 = pulp.lpSum([cfus[i]*x[i][j][k].varValue for (i,k) in product(target_groups,GROUPS)])
    print(f'CFUs for year {j} semester 1: {cfus_sem_1}')
    
    target_groups = [i for i in COURSES if semester[i] == 2]
    cfus_sem_2 = pulp.lpSum([cfus[i]*x[i][j][k].varValue for (i,k) in product(target_groups,GROUPS)])
    print(f'CFUs for year {j} semester 2: {cfus_sem_2}')

100% di REAL AND FUNCTIONAL ANALYSIS [MST, 8 CFU] all'anno 1 al semestre 1
100% di STOCHASTIC DYNAMICAL MODELS [MST, 8 CFU] all'anno 1 al semestre 1
100% di ALGORITHMS AND PARALLEL COMPUTING [MST, 10 CFU] all'anno 1 al semestre 1
100% di MODEL IDENTIFICATION AND DATA ANALYSIS [MST, 10 CFU] all'anno 1 al semestre 2
100% di APPLIED STATISTICS [MST, 10 CFU] all'anno 1 al semestre 2
100% di BAYESIAN STATISTICS [MST, 10 CFU] all'anno 2 al semestre 1
100% di FINAL WORK [MST, 12 CFU] all'anno 2 al semestre 2
100% di COMPUTATIONAL STATISTICS [FREE, 8 CFU] all'anno 1 al semestre 1
100% di DESIGN AND ANALYSIS OF EXPERIMENTS B [STAT, 10 CFU] all'anno 2 al semestre 1
100% di GAME THEORY [FREE, 8 CFU] all'anno 2 al semestre 1
100% di MACHINE LEARNING [ING, 5 CFU] all'anno 2 al semestre 2
100% di ONLINE LEARNING APPLICATIONS [ING, 5 CFU] all'anno 1 al semestre 2
100% di OPTIMIZATION [MTM, 8 CFU] all'anno 2 al semestre 2
100% di REACTION-DIFFUSION EQUATIONS [MTM, 8 CFU] all'anno 1 al semestre 2
     